In [1]:
import torch

BATCH_SIZE = 6 # increase / decrease according to GPU memeory
RESIZE_TO = 512 # resize the image for training and transforms
NUM_EPOCHS = 10 # number of epochs to train for

DEVICE = torch.device('cpu')

# training images and XML files directory
TRAIN_DIR = '../data/rcnn/training'
# validation images and XML files directory
VALID_DIR = '../data/rcnn/validation'

# classes: 0 index is reserved for background
CLASSES = [
    'background', 'abw', 'pbw'
]
NUM_CLASSES = 3

# whether to visualize images after crearing the data loaders
VISUALIZE_TRANSFORMED_IMAGES = False

# location to save model and plots
OUT_DIR = '../results/rcnn'
SAVE_PLOTS_EPOCH = 2 # save loss plots after these many epochs
SAVE_MODEL_EPOCH = 2 # save model after these many epochs

In [2]:
import albumentations as A
import cv2
import numpy as np

from albumentations.pytorch import ToTensorV2

# this class keeps track of the training and validation loss values...
# ... and helps to get the average for each epoch as well
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0
        
    def send(self, value):
        self.current_total += value
        self.iterations += 1
    
    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations
    
    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [3]:
def collate_fn(batch):
    """
    To handle the data loading as different images may have different number 
    of objects and to handle varying size tensors as well.
    """
    return tuple(zip(*batch))

In [4]:
# define the training tranforms
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        A.RandomRotate90(0.5),
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc',
        'label_fields': ['labels']
    })

# define the validation transforms
def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc', 
        'label_fields': ['labels']
    })

# define the negative sample transforms
def get_negative_sample_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ])

In [5]:
def show_tranformed_image(train_loader):
    """
    This function shows the transformed images from the `train_loader`.
    Helps to check whether the tranformed images along with the corresponding
    labels are correct or not.
    Only runs if `VISUALIZE_TRANSFORMED_IMAGES = True` in config.py.
    """
    if len(train_loader) > 0:
        for i in range(1):
            images, targets = next(iter(train_loader))
            images = list(image.to(DEVICE) for image in images)
            targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
            boxes = targets[i]['boxes'].cpu().numpy().astype(np.int32)
            sample = images[i].permute(1, 2, 0).cpu().numpy()
            for box in boxes:
                cv2.rectangle(sample,
                            (box[0], box[1]),
                            (box[2], box[3]),
                            (0, 0, 255), 2)
            cv2.imshow('Transformed image', sample)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

In [6]:
import torch
import cv2
import numpy as np
import os
import glob as glob
import pandas as pd

from torch.utils.data import Dataset, DataLoader

In [7]:
# the dataset class
class WormDataset(Dataset):
    def __init__(self, dir_path, width, height, classes, transforms, negative_sample_transforms):
        self.transforms = transforms
        self.negative_sample_transforms = negative_sample_transforms
        self.dir_path = dir_path
        self.height = height
        self.width = width
        self.classes = classes
        
        # get all the image paths in sorted order
        self.image_paths = glob.glob(f"{self.dir_path}/*[jpg|jpeg]")
        self.all_images = [image_path.split('/')[-1] for image_path in self.image_paths]
        self.all_images = sorted(self.all_images)

    def __getitem__(self, idx):
        # capture the image name and the full image path
        image_name = self.all_images[idx]
        image_path = os.path.join(self.dir_path, image_name)

        # read the image
        image = cv2.imread(image_path)
        # convert BGR to RGB color format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image_resized = cv2.resize(image, (self.width, self.height))
        image_resized /= 255.0
        
        # capture the corresponding XML file for getting the annotations
        annotations_file = image_name.split('.')[0] + '.txt'
        annotations_file_path = os.path.join(self.dir_path, annotations_file)
        
        boxes = []
        labels = []
        
        # get the height and width of the image
        image_width = image.shape[1]
        image_height = image.shape[0]
        
        # box coordinates for xml files are extracted and corrected for image size given

        bounding_boxes_df = pd.read_csv(annotations_file_path, sep='\t', names=["target", 'xmin', 'ymin', 'xmax', 'ymax'])
        bounding_boxes_df['xmin'] = bounding_boxes_df['xmin'] * self.width
        bounding_boxes_df['xmax'] = bounding_boxes_df['xmax'] * self.width
        bounding_boxes_df['ymin'] = bounding_boxes_df['ymin'] * self.height
        bounding_boxes_df['ymax'] = bounding_boxes_df['ymax'] * self.height
        
        # bounding box to tensor
        boxes = bounding_boxes_df[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()
        labels = bounding_boxes_df['target'].to_numpy()
        
        is_negative_sample = len(boxes) == 0

        if is_negative_sample:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.as_tensor(np.array([0]), dtype=torch.int64)
        else:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            # labels to tensor
            labels = torch.as_tensor(labels, dtype=torch.int64)

        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)

        # prepare the final `target` dictionary
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd
        image_id = torch.tensor([idx])
        target["image_id"] = image_id

        # apply the image transforms
        if not is_negative_sample:
            sample = self.transforms(image = image_resized,
                                     bboxes = target['boxes'],
                                     labels = labels)
            image_resized = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
        elif is_negative_sample:
            sample = self.negative_sample_transforms(image=image_resized,
                                                     labels = labels)
            image_resized = sample['image']
        
        return image_resized, target

    def __len__(self):
        return len(self.all_images)

In [8]:
# prepare the final datasets and data loaders
train_dataset = WormDataset(TRAIN_DIR, RESIZE_TO, RESIZE_TO, CLASSES, get_train_transform(), get_negative_sample_transform())
valid_dataset = WormDataset(VALID_DIR, RESIZE_TO, RESIZE_TO, CLASSES, get_valid_transform(), get_negative_sample_transform())
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(valid_dataset)}\n")

Number of training samples: 8763
Number of validation samples: 974



In [9]:
# # sanity check of the Dataset pipeline with sample visualization
# dataset = WormDataset(
#     TRAIN_DIR, RESIZE_TO, RESIZE_TO, CLASSES
# )
# print(f"Number of training images: {len(dataset)}")

# # function to visualize a single sample
# def visualize_sample(image, target):
#     box = target['boxes'][0] if len(target['boxes']) > 0 else []
#     label = CLASSES[target['labels'][0]] if len(target['labels']) > 0 else []
#     if len(box) != 0:
#         cv2.rectangle(
#             image, 
#             (int(box[0]), int(box[1])), (int(box[2]), int(box[3])),
#             (0, 255, 0), 1
#         )
#         cv2.putText(
#             image, label, (int(box[0]), int(box[1]-5)), 
#             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2
#         )
#     cv2.imshow('Image', image)
#     cv2.waitKey(0)
    
# NUM_SAMPLES_TO_VISUALIZE = 5
# for i in range(NUM_SAMPLES_TO_VISUALIZE):
#     image, target = dataset[i]
#     visualize_sample(image, target)

In [10]:
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN_ResNet50_FPN_Weights

def create_model(num_classes):
    
    # load Faster RCNN pre-trained model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
    
    # get the number of input features 
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # define a new head for the detector with required number of classes
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 

    return model

In [11]:
from tqdm.auto import tqdm

import torch
import matplotlib.pyplot as plt
import time

plt.style.use('ggplot')

In [12]:
# function for running training iterations
def train(train_data_loader, model, optimizer, train_loss_hist, train_loss_list):
    print('Training')
    
    for i, data in enumerate(train_data_loader):
        optimizer.zero_grad()
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        train_loss_list.append(loss_value)

        train_loss_hist.send(loss_value)

        losses.backward()
        optimizer.step()
    
        # update the loss value beside the progress bar for each iteration
        print(f"Loss: {loss_value:.4f} ,Index: {i}, Total: {len(train_data_loader)}")
    return train_loss_list

In [13]:
# function for running validation iterations
def validate(valid_data_loader, model, val_loss_hist, val_loss_list):
    print('Validating')
    
    for i, data in enumerate(valid_data_loader):
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        
        with torch.no_grad():
            loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        val_loss_list.append(loss_value)

        val_loss_hist.send(loss_value)

        # update the loss value beside the progress bar for each iteration
        print(f"Loss: {loss_value:.4f} ,Index: {i}, Total: {len(valid_data_loader)}")
    return val_loss_list

In [14]:
# initialize the model and move to the computation device
model = create_model(num_classes=NUM_CLASSES)
model = model.to(DEVICE)
# get the model parameters
params = [p for p in model.parameters() if p.requires_grad]
# define the optimizer
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)

# initialize the Averager class
train_loss_hist = Averager()
val_loss_hist = Averager()
# train and validation loss lists to store loss values of all...
# ... iterations till ena and plot graphs for all iterations
train_loss_list = []
val_loss_list = []

# name to save the trained model with
MODEL_NAME = 'rcnn'

# whether to show transformed images from data loader or not
if VISUALIZE_TRANSFORMED_IMAGES:
    show_tranformed_image(train_loader)

# start the training epochs
for epoch in range(NUM_EPOCHS):
    print(f"\nEPOCH {epoch+1} of {NUM_EPOCHS}")

    # reset the training and validation loss histories for the current epoch
    train_loss_hist.reset()
    val_loss_hist.reset()

    # create two subplots, one for each, training and validation
    figure_1, train_ax = plt.subplots()
    figure_2, valid_ax = plt.subplots()

    # start timer and carry out training and validation
    start = time.time()
    train_loss = train(train_loader, model, optimizer, train_loss_hist, train_loss_list)
    val_loss = validate(valid_loader, model, val_loss_hist, val_loss_list)
    print(f"Epoch #{epoch} train loss: {train_loss_hist.value:.3f}")   
    print(f"Epoch #{epoch} validation loss: {val_loss_hist.value:.3f}")   
    end = time.time()
    print(f"Took {((end - start) / 60):.3f} minutes for epoch {epoch}")

    if (epoch+1) % SAVE_MODEL_EPOCH == 0: # save model after every n epochs
        torch.save(model.state_dict(), f"{OUT_DIR}/model{epoch+1}.pth")
        print('SAVING MODEL COMPLETE...\n')
    
    if (epoch+1) % SAVE_PLOTS_EPOCH == 0: # save loss plots after n epochs
        train_ax.plot(train_loss, color='blue')
        train_ax.set_xlabel('iterations')
        train_ax.set_ylabel('train loss')
        valid_ax.plot(val_loss, color='red')
        valid_ax.set_xlabel('iterations')
        valid_ax.set_ylabel('validation loss')
        figure_1.savefig(f"{OUT_DIR}/train_loss_{epoch+1}.png")
        figure_2.savefig(f"{OUT_DIR}/valid_loss_{epoch+1}.png")
        print('SAVING PLOTS COMPLETE...')
    
    if (epoch+1) == NUM_EPOCHS: # save loss plots and model once at the end
        train_ax.plot(train_loss, color='blue')
        train_ax.set_xlabel('iterations')
        train_ax.set_ylabel('train loss')
        valid_ax.plot(val_loss, color='red')
        valid_ax.set_xlabel('iterations')
        valid_ax.set_ylabel('validation loss')
        figure_1.savefig(f"{OUT_DIR}/train_loss_{epoch+1}.png")
        figure_2.savefig(f"{OUT_DIR}/valid_loss_{epoch+1}.png")

        torch.save(model.state_dict(), f"{OUT_DIR}/model{epoch+1}.pth")
    
    plt.close('all')


EPOCH 1 of 10
Training
Loss: 1.8108 ,Index: 0, Total: 1461
Loss: 3.3769 ,Index: 1, Total: 1461
Loss: 1.8650 ,Index: 2, Total: 1461
Loss: 1.0190 ,Index: 3, Total: 1461
Loss: 1.2924 ,Index: 4, Total: 1461
Loss: 0.8862 ,Index: 5, Total: 1461
Loss: 0.9238 ,Index: 6, Total: 1461
Loss: 1.5057 ,Index: 7, Total: 1461
Loss: 0.5452 ,Index: 8, Total: 1461
Loss: 0.7229 ,Index: 9, Total: 1461
Loss: 0.8082 ,Index: 10, Total: 1461
Loss: 0.8799 ,Index: 11, Total: 1461
Loss: 1.4901 ,Index: 12, Total: 1461
Loss: 0.9044 ,Index: 13, Total: 1461
Loss: 0.5910 ,Index: 14, Total: 1461
Loss: 0.5190 ,Index: 15, Total: 1461
Loss: 0.7643 ,Index: 16, Total: 1461
Loss: 0.6880 ,Index: 17, Total: 1461
Loss: 0.8515 ,Index: 18, Total: 1461
Loss: 0.6470 ,Index: 19, Total: 1461
Loss: 0.7878 ,Index: 20, Total: 1461
Loss: 0.6178 ,Index: 21, Total: 1461
Loss: 0.6906 ,Index: 22, Total: 1461
Loss: 0.6277 ,Index: 23, Total: 1461
Loss: 1.2652 ,Index: 24, Total: 1461
Loss: 0.2259 ,Index: 25, Total: 1461
Loss: 0.9204 ,Index: 26,

Invalid SOS parameters for sequential JPEG


Loss: 0.5441 ,Index: 114, Total: 1461
Loss: 0.7754 ,Index: 115, Total: 1461
Loss: 0.7265 ,Index: 116, Total: 1461
Loss: 0.4627 ,Index: 117, Total: 1461
Loss: 0.3768 ,Index: 118, Total: 1461
Loss: 0.4862 ,Index: 119, Total: 1461
Loss: 0.6284 ,Index: 120, Total: 1461
Loss: 0.2111 ,Index: 121, Total: 1461
Loss: 0.4070 ,Index: 122, Total: 1461
Loss: 0.3801 ,Index: 123, Total: 1461
Loss: 0.4369 ,Index: 124, Total: 1461
Loss: 0.3150 ,Index: 125, Total: 1461
Loss: 0.2997 ,Index: 126, Total: 1461
Loss: 0.4770 ,Index: 127, Total: 1461
Loss: 0.3673 ,Index: 128, Total: 1461
Loss: 0.0758 ,Index: 129, Total: 1461
Loss: 0.5458 ,Index: 130, Total: 1461
Loss: 0.7186 ,Index: 131, Total: 1461
Loss: 0.4529 ,Index: 132, Total: 1461
Loss: 0.7315 ,Index: 133, Total: 1461
Loss: 0.3923 ,Index: 134, Total: 1461
Loss: 0.4079 ,Index: 135, Total: 1461
Loss: 0.3343 ,Index: 136, Total: 1461
Loss: 0.2539 ,Index: 137, Total: 1461
Loss: 0.5085 ,Index: 138, Total: 1461
Loss: 0.9015 ,Index: 139, Total: 1461
Loss: 0.7137

Invalid SOS parameters for sequential JPEG


Loss: 0.2630 ,Index: 304, Total: 1461
Loss: 0.7932 ,Index: 305, Total: 1461
Loss: 0.5371 ,Index: 306, Total: 1461
Loss: 0.1861 ,Index: 307, Total: 1461
Loss: 0.3125 ,Index: 308, Total: 1461
Loss: 0.5783 ,Index: 309, Total: 1461
Loss: 0.4901 ,Index: 310, Total: 1461
Loss: 0.4441 ,Index: 311, Total: 1461
Loss: 0.6501 ,Index: 312, Total: 1461
Loss: 0.4914 ,Index: 313, Total: 1461
Loss: 0.6408 ,Index: 314, Total: 1461
Loss: 0.3863 ,Index: 315, Total: 1461
Loss: 0.2353 ,Index: 316, Total: 1461
Loss: 0.9242 ,Index: 317, Total: 1461
Loss: 0.2781 ,Index: 318, Total: 1461
Loss: 0.4242 ,Index: 319, Total: 1461
Loss: 0.6796 ,Index: 320, Total: 1461
Loss: 0.2165 ,Index: 321, Total: 1461
Loss: 0.1730 ,Index: 322, Total: 1461
Loss: 0.9959 ,Index: 323, Total: 1461
Loss: 0.8985 ,Index: 324, Total: 1461
Loss: 0.7292 ,Index: 325, Total: 1461
Loss: 0.3599 ,Index: 326, Total: 1461
Loss: 0.5219 ,Index: 327, Total: 1461
Loss: 0.2843 ,Index: 328, Total: 1461
Loss: 0.7384 ,Index: 329, Total: 1461
Loss: 0.3043

...

Loss: 0.4677 ,Index: 851, Total: 1461
Loss: 0.3746 ,Index: 852, Total: 1461
Loss: 0.3256 ,Index: 853, Total: 1461
Loss: 0.7019 ,Index: 854, Total: 1461
Loss: 0.4521 ,Index: 855, Total: 1461
Loss: 0.6372 ,Index: 856, Total: 1461
Loss: 0.0679 ,Index: 857, Total: 1461
Loss: 0.0850 ,Index: 858, Total: 1461
Loss: 0.4052 ,Index: 859, Total: 1461
Loss: 0.5156 ,Index: 860, Total: 1461
Loss: 0.3296 ,Index: 861, Total: 1461
Loss: 0.4309 ,Index: 862, Total: 1461
Loss: 0.6484 ,Index: 863, Total: 1461
Loss: 0.5331 ,Index: 864, Total: 1461
Loss: 0.1833 ,Index: 865, Total: 1461
Loss: 0.3623 ,Index: 866, Total: 1461
Loss: 0.3936 ,Index: 867, Total: 1461
Loss: 0.5458 ,Index: 868, Total: 1461
Loss: 0.6198 ,Index: 869, Total: 1461
Loss: 0.2709 ,Index: 870, Total: 1461
Loss: 0.5864 ,Index: 871, Total: 1461
Loss: 0.5257 ,Index: 872, Total: 1461
Loss: 0.5586 ,Index: 873, Total: 1461
Loss: 0.2396 ,Index: 874, Total: 1461
Loss: 0.2205 ,Index: 875, Total: 1461
Loss: 0.5382 ,Index: 876, Total: 1461
Loss: 0.1695

Invalid SOS parameters for sequential JPEG


Loss: 0.4006 ,Index: 920, Total: 1461
Loss: 0.2577 ,Index: 921, Total: 1461
Loss: 0.2782 ,Index: 922, Total: 1461
Loss: 0.3113 ,Index: 923, Total: 1461
Loss: 0.4212 ,Index: 924, Total: 1461
Loss: 0.2505 ,Index: 925, Total: 1461
Loss: 0.0723 ,Index: 926, Total: 1461
Loss: 0.3359 ,Index: 927, Total: 1461
Loss: 0.2187 ,Index: 928, Total: 1461
Loss: 0.2258 ,Index: 929, Total: 1461
Loss: 0.4825 ,Index: 930, Total: 1461
Loss: 0.3143 ,Index: 931, Total: 1461
Loss: 0.3404 ,Index: 932, Total: 1461
Loss: 0.3875 ,Index: 933, Total: 1461
Loss: 0.2391 ,Index: 934, Total: 1461
Loss: 0.6810 ,Index: 935, Total: 1461
Loss: 0.3353 ,Index: 936, Total: 1461
Loss: 0.2523 ,Index: 937, Total: 1461
Loss: 0.7014 ,Index: 938, Total: 1461
Loss: 0.6746 ,Index: 939, Total: 1461
Loss: 0.2244 ,Index: 940, Total: 1461
Loss: 0.2653 ,Index: 941, Total: 1461
Loss: 0.4695 ,Index: 942, Total: 1461
Loss: 0.5967 ,Index: 943, Total: 1461
Loss: 0.4504 ,Index: 944, Total: 1461
Loss: 0.5235 ,Index: 945, Total: 1461
Loss: 0.3658

Invalid SOS parameters for sequential JPEG


Loss: 0.4970 ,Index: 14, Total: 163
Loss: 0.1262 ,Index: 15, Total: 163
Loss: 0.2799 ,Index: 16, Total: 163
Loss: 0.2448 ,Index: 17, Total: 163
Loss: 0.4762 ,Index: 18, Total: 163
Loss: 0.2477 ,Index: 19, Total: 163
Loss: 0.1850 ,Index: 20, Total: 163
Loss: 0.3738 ,Index: 21, Total: 163
Loss: 0.2366 ,Index: 22, Total: 163
Loss: 0.5966 ,Index: 23, Total: 163
Loss: 0.2313 ,Index: 24, Total: 163
Loss: 0.1897 ,Index: 25, Total: 163
Loss: 0.5664 ,Index: 26, Total: 163
Loss: 0.2349 ,Index: 27, Total: 163
Loss: 0.3052 ,Index: 28, Total: 163
Loss: 0.3442 ,Index: 29, Total: 163
Loss: 0.7272 ,Index: 30, Total: 163
Loss: 0.1985 ,Index: 31, Total: 163
Loss: 0.6937 ,Index: 32, Total: 163
Loss: 0.3263 ,Index: 33, Total: 163
Loss: 0.1652 ,Index: 34, Total: 163
Loss: 0.1825 ,Index: 35, Total: 163
Loss: 0.5156 ,Index: 36, Total: 163
Loss: 0.4025 ,Index: 37, Total: 163
Loss: 0.4625 ,Index: 38, Total: 163
Loss: 0.1903 ,Index: 39, Total: 163
Loss: 0.5844 ,Index: 40, Total: 163
Loss: 0.3750 ,Index: 41, Tot

In [15]:
import numpy as np
import cv2
import torch
import glob as glob

# set the computation device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# load the model and the trained weights
model = create_model(num_classes=NUM_CLASSES).to(device)
model.load_state_dict(torch.load(
    f'{OUT_DIR}/model10.pth', map_location=device
))
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [35]:
# directory where all the images are present
DIR_TEST = '../data/rcnn/testing/'
test_images = glob.glob(f"{DIR_TEST}/*")
print(f"Test instances: {len(test_images)}")

# define the detection threshold...
# ... any detection having score below this will be discarded
detection_threshold = 0.5

Test instances: 2803


In [36]:
results = []
for i in range(len(test_images)):
    # get the image file name for saving output later on
    image_name = test_images[i].split('/')[-1].split('.')[0]
    image = cv2.imread(test_images[i])
    orig_image = image.copy()
    # BGR to RGB
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
    # make the pixel range between 0 and 1
    image /= 255.0
    # bring color channels to front
    image = np.transpose(image, (2, 0, 1)).astype(np.float)
    # convert to tensor
    image = torch.tensor(image, dtype=torch.float)
    # add batch dimension
    image = torch.unsqueeze(image, 0)
    with torch.no_grad():
        outputs = model(image)
    
    # load all detection to CPU for further operations
    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
    # carry further only if there are detected boxes
    if len(outputs[0]['boxes']) != 0:
        boxes = outputs[0]['boxes'].data.numpy()
        scores = outputs[0]['scores'].data.numpy()
        # filter out boxes according to `detection_threshold`
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        draw_boxes = boxes.copy()
        # get all the predicted class names
        pred_classes = [CLASSES[i] for i in outputs[0]['labels'].numpy()]
        
        # draw the bounding boxes and write the class name on top of it
        for j, box in enumerate(draw_boxes):
            cv2.rectangle(orig_image,
                        (int(box[0]), int(box[1])),
                        (int(box[2]), int(box[3])),
                        (0, 0, 255), 2)
            cv2.putText(orig_image, pred_classes[j], 
                        (int(box[0]), int(box[1]-5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 
                        2, lineType=cv2.LINE_AA)
            results.append({
                "image": test_images[i].split('/')[-1],
                "bounding_box": box,
                "prediction": pred_classes[j]
            })

        # cv2.imshow('Prediction', orig_image)
        # cv2.waitKey(1)
        cv2.imwrite(f"{OUT_DIR}/test_predictions/{image_name}.jpg", orig_image,)
    print(f"Image {i+1} done...")
    print('-'*50)

/var/folders/n2/bwdv1zhn55ngx2l75t_7jwkw0000gn/T/ipykernel_84523/1240347181.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  image = np.transpose(image, (2, 0, 1)).astype(np.float)


Image 1 done...
--------------------------------------------------
Image 2 done...
--------------------------------------------------
Image 3 done...
--------------------------------------------------
Image 4 done...
--------------------------------------------------
Image 5 done...
--------------------------------------------------
Image 6 done...
--------------------------------------------------
Image 7 done...
--------------------------------------------------
Image 8 done...
--------------------------------------------------
Image 9 done...
--------------------------------------------------
Image 10 done...
--------------------------------------------------
Image 11 done...
--------------------------------------------------
Image 12 done...
--------------------------------------------------
Image 13 done...
--------------------------------------------------
Image 14 done...
--------------------------------------------------
Image 15 done...
----------------------------------------

Invalid SOS parameters for sequential JPEG


Image 928 done...
--------------------------------------------------
Image 929 done...
--------------------------------------------------
Image 930 done...
--------------------------------------------------
Image 931 done...
--------------------------------------------------
Image 932 done...
--------------------------------------------------
Image 933 done...
--------------------------------------------------
Image 934 done...
--------------------------------------------------
Image 935 done...
--------------------------------------------------
Image 936 done...
--------------------------------------------------
Image 937 done...
--------------------------------------------------
Image 938 done...
--------------------------------------------------
Image 939 done...
--------------------------------------------------
Image 940 done...
--------------------------------------------------
Image 941 done...
--------------------------------------------------
Image 942 done...
----------------

In [37]:
import pandas as pd

rcnn_results_df = pd.DataFrame(results)
rcnn_results_df

,image,bounding_box,prediction
0,id_3b59b5876e0a54fd5fd38188.jpg,"[513, 734, 543, 755]",pbw
1,id_3b59b5876e0a54fd5fd38188.jpg,"[322, 167, 353, 193]",pbw
2,id_3b59b5876e0a54fd5fd38188.jpg,"[654, 825, 680, 854]",pbw
3,id_3b59b5876e0a54fd5fd38188.jpg,"[537, 742, 565, 762]",pbw
4,id_3b59b5876e0a54fd5fd38188.jpg,"[320, 603, 352, 621]",pbw
...,...,...,...
39943,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[1470, 211, 1911, 856]",abw
39944,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[2311, 2132, 2918, 3006]",abw
39945,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[1304, 1615, 1822, 2438]",abw
39946,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[451, 50, 838, 785]",abw


In [38]:
rcnn_predictions_df = rcnn_results_df.groupby(['image', 'prediction']).size().reset_index(name='target')
rcnn_predictions_df['image'] = rcnn_predictions_df['image'].apply(lambda x: os.path.splitext(x)[0])
rcnn_predictions_df['Image_ID'] = (rcnn_predictions_df['image'] + "_" + rcnn_predictions_df['prediction']).str.lower()
rcnn_predictions_df = rcnn_predictions_df.drop(columns=['image', 'prediction'])
rcnn_predictions_df

,target,Image_ID
0,5,id_00332970f80fa9a47a39516d_abw
1,21,id_0035981bc3ae42eb5b57a317_pbw
2,9,id_005102f664b820f778291dee_abw
3,5,id_0066456f5fb2cd858c69ab39_abw
4,8,id_00887bebda26184c36e18e00_pbw
...,...,...
2327,11,id_ff40ad3bb245a697d799eeef_abw
2328,1,id_ff6b60935a327d1c2779bc5e_pbw
2329,7,id_ffad8f3773a4222f8fe5ba1a_pbw
2330,98,id_ffb65e6de900c49d8f2ef95a_pbw


In [39]:
test_df = pd.read_csv('../data/test.csv')

abw_df = test_df.copy(deep=True)
abw_df['worm_type'] = 'abw'
pbw_df = test_df.copy(deep=True)
pbw_df['worm_type'] = 'pbw'

full_test_df = pd.concat([abw_df, pbw_df], ignore_index=True)
full_test_df['image_id_worm'] = full_test_df['image_id_worm'].apply(lambda x: os.path.splitext(x)[0])
full_test_df['image_id_worm'] = (full_test_df['image_id_worm'] + "_" + full_test_df['worm_type']).str.lower()
full_test_df = full_test_df.drop(columns=['worm_type'])
full_test_df = full_test_df.rename(columns={"image_id_worm": "Image_ID"})
full_test_df

,Image_ID
0,id_00332970f80fa9a47a39516d_abw
1,id_0035981bc3ae42eb5b57a317_abw
2,id_005102f664b820f778291dee_abw
3,id_0066456f5fb2cd858c69ab39_abw
4,id_007159c1fa015ba6f394deeb_abw
...,...
5601,id_ffad8f3773a4222f8fe5ba1a_pbw
5602,id_ffb65e6de900c49d8f2ef95a_pbw
5603,id_ffbcb27fa549278f47505515_pbw
5604,id_ffc0e41e10b0c964d4a02811_pbw


In [40]:
final_testing_df = rcnn_predictions_df.copy(deep=True)

final_testing_df = pd.merge(full_test_df, final_testing_df, on="Image_ID", how="left")

final_testing_df['target'] = final_testing_df['target'].fillna(0)
final_testing_df['target'] = pd.to_numeric(final_testing_df['target'], downcast='integer')
final_testing_df = final_testing_df.sort_values(by='Image_ID').reset_index(drop=True)

final_testing_df.to_csv('../results/rcnn/rcnn_test_results_1.csv', index=False)
final_testing_df

,Image_ID,target
0,id_00332970f80fa9a47a39516d_abw,5
1,id_00332970f80fa9a47a39516d_pbw,0
2,id_0035981bc3ae42eb5b57a317_abw,0
3,id_0035981bc3ae42eb5b57a317_pbw,21
4,id_005102f664b820f778291dee_abw,9
...,...,...
5601,id_ffbcb27fa549278f47505515_pbw,0
5602,id_ffc0e41e10b0c964d4a02811_abw,0
5603,id_ffc0e41e10b0c964d4a02811_pbw,0
5604,id_fff8c253115aacded09ad7ed_abw,0
